In [1]:
import os
import io
import sys
from dotenv import load_dotenv
from openai import OpenAI
from IPython.display import Markdown, display, update_display
import google.generativeai as genai
import gradio as gr
import subprocess
load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')
or_api_key = os.getenv('OR_API_KEY')

In [2]:
openai = OpenAI()
genai.configure(api_key=google_api_key)
google = genai.GenerativeModel("gemini-2.5-flash")
openrouter = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=or_api_key
)

OPENAI_MODEL = "gpt-4o-mini"
GOOGLE_MODEL = "gemini-2.5-pro"
OPENROUTER_MODEL = "deepseek/deepseek-chat-v3.1:free"


In [3]:
system_message = """You are an assistant that reimplements Python code in high performance rust for an AMD RYZEN 5 7520u in ubuntu 22 lts.
Respond only with Rust code; use comments sparingly and do not provide any explanation of any thing. If the Python code uses any libraries, use equivalent Rust libraries.
The Rust response needs to produce an identical output in the fastest possible time it run on ubuntu 22 lts machine. write the code in main.rs only"""

In [4]:
def user_prompt_for(python):
    user_prompt = """Rewrite this Python code in Rust with the fastest possible implementation that produces identical output in the least time.
Respond only with Rust code; do not explain your work or provide any commentary.
Pay attention to number types to ensure no int overflows. Remember to import all necessary Rust crates such as std::io, std::collections, etc.
Here Below is the Python code:"""
    user_prompt += python
    return user_prompt

In [5]:
def messages_for(python):
    return [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_prompt_for(python)}
    ]

def write_output(rust):
    code = rust.replace("```rust","").replace("```","")
    with open("main.rs", "w") as f:
        f.write(code)

In [6]:
def optimize_gpt(python):    
    stream = openai.chat.completions.create(model=OPENAI_MODEL, messages=messages_for(python), stream=True)
    reply = ""
    for chunk in stream:
        fragment = chunk.choices[0].delta.content or ""
        reply += fragment
        print(fragment, end='', flush=True)
    write_output(reply)

In [7]:
def optimize_openrouter(python):    
    stream = openrouter.chat.completions.create(model=OPENAI_MODEL, messages=messages_for(python), stream=True)
    reply = ""
    for chunk in stream:
        fragment = chunk.choices[0].delta.content or ""
        reply += fragment
        print(fragment, end='', flush=True)
    write_output(reply)

In [8]:
def optimize_google(python):
    messages = messages_for(python)
    messages.append({'role':'system','parts': messages[0]['content']})
    messages.pop(0)
    messages.append({'role':'user','parts': messages[0]['content']+system_message})
    messages.pop(0)
    messages.pop(0)
    stream = google.generate_content(messages)
    reply = ""
    for chunk in stream:
        fragment = chunk.text or ""
        reply += fragment
        print(fragment, end='', flush=True)
    write_output(reply)

In [9]:
pi = """
import time

def calculate(iterations, param1, param2):
    result = 1.0
    for i in range(1, iterations+1):
        j = i * param1 - param2
        result -= (1/j)
        j = i * param1 + param2
        result += (1/j)
    return result

start_time = time.time()
result = calculate(100_000_000, 4, 1) * 4
end_time = time.time()

print(f"Result: {result:.12f}")
print(f"Execution Time: {(end_time - start_time):.6f} seconds")
"""

In [10]:
optimize_gpt(pi)

```rust
use std::time::Instant;

fn calculate(iterations: u64, param1: f64, param2: f64) -> f64 {
    let mut result = 1.0;
    for i in 1..=iterations {
        let j1 = i as f64 * param1 - param2;
        result -= 1.0 / j1;
        let j2 = i as f64 * param1 + param2;
        result += 1.0 / j2;
    }
    result
}

fn main() {
    let start_time = Instant::now();
    let result = calculate(100_000_000, 4.0, 1.0) * 4.0;
    let end_time = Instant::now();

    println!("Result: {:.12}", result);
    println!("Execution Time: {:.6} seconds", (end_time - start_time).as_secs_f64());
}
```

In [11]:
exec(pi)

Result: 3.141592658589
Execution Time: 15.121928 seconds


In [12]:
!rustc -C opt-level=3 -C target-cpu=native -C lto main.rs -o optimized_rust && ./optimized_rust


Result: 3.141592658589
Execution Time: 0.517851 seconds


In [34]:
optimize_google(pi)

```java
import java.text.DecimalFormat;

public class Main {

    public static double calculate(int iterations, int param1, int param2) {
        double result = 1.0;
        for (int i = 1; i <= iterations; i++) {
            // These intermediate calculations fit within int range
            // (100,000,000 * 4) is 400,000,000, well within Integer.MAX_VALUE (~2.1 billion)
            int jSub = i * param1 - param2;
            result -= (1.0 / jSub); // Use 1.0 for double division

            int jAdd = i * param1 + param2;
            result += (1.0 / jAdd); // Use 1.0 for double division
        }
        return result;
    }

    public static void main(String[] args) {
        long startTime = System.nanoTime();

        double calculationResult = calculate(100_000_000, 4, 1) * 4;

        long endTime = System.nanoTime();

        // Format the output to match Python's f-string precision (12 decimal places)
        DecimalFormat resultFormatter = new DecimalFormat("0.000000000

In [17]:
optimize_openrouter(pi)


```rust
use std::time::Instant;

fn calculate(iterations: u64, param1: f64, param2: f64) -> f64 {
    let mut result = 1.0;
    for i in 1..=iterations {
        let j1 = i as f64 * param1 - param2;
        result -= 1.0 / j1;
        let j2 = i as f64 * param1 + param2;
        result += 1.0 / j2;
    }
    result
}

fn main() {
    let start_time = Instant::now();
    let result = calculate(100_000_000, 4.0, 1.0) * 4.0;
    let end_time = Instant::now();

    println!("Result: {:.12}", result);
    println!("Execution Time: {:.6} seconds", (end_time - start_time).as_secs_f64());
}
```

In [14]:
python_hard = """# Be careful to support large number sizes

def lcg(seed, a=1664525, c=1013904223, m=2**32):
    value = seed
    while True:
        value = (a * value + c) % m
        yield value
        
def max_subarray_sum(n, seed, min_val, max_val):
    lcg_gen = lcg(seed)
    random_numbers = [next(lcg_gen) % (max_val - min_val + 1) + min_val for _ in range(n)]
    max_sum = float('-inf')
    for i in range(n):
        current_sum = 0
        for j in range(i, n):
            current_sum += random_numbers[j]
            if current_sum > max_sum:
                max_sum = current_sum
    return max_sum

def total_max_subarray_sum(n, initial_seed, min_val, max_val):
    total_sum = 0
    lcg_gen = lcg(initial_seed)
    for _ in range(20):
        seed = next(lcg_gen)
        total_sum += max_subarray_sum(n, seed, min_val, max_val)
    return total_sum

# Parameters
n = 10000         # Number of random numbers
initial_seed = 42 # Initial seed for the LCG
min_val = -10     # Minimum value of random numbers
max_val = 10      # Maximum value of random numbers

# Timing the function
import time
start_time = time.time()
result = total_max_subarray_sum(n, initial_seed, min_val, max_val)
end_time = time.time()

print("Total Maximum Subarray Sum (20 runs):", result)
print("Execution Time: {:.6f} seconds".format(end_time - start_time))
"""

In [18]:
!rustc -C opt-level=3 -C target-cpu=native -C lto main.rs -o optimized_rust && ./optimized_rust


Result: 3.141592658589
Execution Time: 0.516361 seconds


In [19]:
def stream_gpt(python):    
    stream = openai.chat.completions.create(model=OPENAI_MODEL, messages=messages_for(python), stream=True)
    reply = ""
    for chunk in stream:
        fragment = chunk.choices[0].delta.content or ""
        reply += fragment
        yield reply.replace('```rust\n','').replace('```','')

In [20]:
def stream_openrouter(python):    
    stream = openrouter.chat.completions.create(model=OPENAI_MODEL, messages=messages_for(python), stream=True)
    reply = ""
    for chunk in stream:
        fragment = chunk.choices[0].delta.content or ""
        reply += fragment
        yield reply.replace('```rust\n','').replace('```','')

In [21]:
def stream_google(python_code: str):

    messages = [
        {"role": "user", "parts": user_prompt_for(python_code)}
    ]

    # Start streaming
    stream = google.generate_content(messages, stream=True)
    reply = ""

    for chunk in stream:
        if chunk.text:   # Only add non-empty parts
            reply += chunk.text
            yield reply.replace("```rust\n", "").replace("```", "")

In [22]:
for output in stream_google(pi):
    print(output, end='\r', flush=True)

use std::time::Instant;

fn calculate(iterations: u64, param1_int: i64, param2_int: i64) -> f64 {
use std::time::Instant;

fn calculate(iterations: u64, param1_int: i64, param2_int: i64) -> f64 {
    let mut result: f64 = 1.0;
    
    // Convert integer parameters to f64 once to avoid repeated casting inside the loop.
    let param1_f64 = param1_int as f64;
use std::time::Instant;

fn calculate(iterations: u64, param1_int: i64, param2_int: i64) -> f64 {
    let mut result: f64 = 1.0;
    
    // Convert integer parameters to f64 once to avoid repeated casting inside the loop.
    let param1_f64 = param1_int as f64;
    let param2_f64 = param2_int as f64;

    // Iterate from 1 up to and including 'iterations'.
    for i in 1..=iterations {
use std::time::Instant;loop

fn calculate(iterations: u64, param1_int: i64, param2_int: i64) -> f64 {
    let mut result: f64 = 1.0;
    
    // Convert integer parameters to f64 once to avoid repeated casting inside the loop.
    let param1_f64 = p

In [28]:
def optimize(python, model):
    if model=="GPT":
        result = stream_gpt(python)
    elif model=="Google":
        result = stream_google(python)
    elif model=="OpenRouter":
        result = stream_openrouter(python)
    else:
        raise ValueError("Unknown model")
    for stream_so_far in result:
        yield stream_so_far     

In [24]:
def execute_python(code):
    try:
        output = io.StringIO()
        sys.stdout = output
        exec(code)
    finally:
        sys.stdout = sys.__stdout__
    return output.getvalue()

In [34]:
def execute_rust(code):
        write_output(code)
        try:
            compile_cmd = "rustc -C opt-level=3 -C target-cpu=native -C lto main.rs -o optimized_gradio".split(' ')
            compile_result = subprocess.run(compile_cmd, check=True, text=True, capture_output=True)
            run_cmd = ["./optimized"]
            run_result = subprocess.run(run_cmd, check=True, text=True, capture_output=True)
            return run_result.stdout
        except subprocess.CalledProcessError as e:
            return f"An error occurred:\n{e.stderr}"

In [26]:
css = """
.python {background-color: #306998;}
.rust {background-color: #050;}
"""

In [35]:
with gr.Blocks(css=css) as ui:
    gr.Markdown("## Convert code from Python to Rust")
    with gr.Row():
        python = gr.Textbox(label="Python code:", value=pi, lines=10)
        rust = gr.Textbox(label="Rust code", lines=10)
    with gr.Row():
        model = gr.Dropdown(["GPT", "Google", "OpenRouter"], label="Select model", value="OpenRouter")
    with gr.Row():
        convert = gr.Button("Convert code")
    with gr.Row():
        python_run = gr.Button("Run Python")
        rust_run = gr.Button("Run Rust")
    with gr.Row():
        python_out = gr.TextArea(label="Python result:", elem_classes=["python"])
        rust_out = gr.TextArea(label="Rust result:", elem_classes=["rust"])

    convert.click(optimize, inputs=[python, model], outputs=[rust])
    python_run.click(execute_python, inputs=[python], outputs=[python_out])
    rust_run.click(execute_rust, inputs=[rust], outputs=[rust_out])

ui.launch(inbrowser=True)

